
<b>Decision Variable:</b>

\begin{equation}
\begin{array}{11}
\ Y_{i,k} & \forall i \in R, k \in WD
\end{array}
\end{equation}
\begin{equation}
\begin{array}{11}
\ X_{i,j,k} & \forall i \in R, j \in S, k \in WD
\end{array}
\end{equation}

<b>Objective Function:</b>

\begin{equation}
\begin{array}{ll}
\text{minimise} &  \sum_{i \in R}  \sum_{k \in W} Y_{i,k}  c_{i,k}\\
\end{array}
\end{equation}


<b>Subject to:</b>
\
<i>Respecting Demand Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{i \in R} \sum_{k \in W}  X_{i,j,k} m_{i,j} = d_j & \forall j \in S
\end{array}
\end{equation}

\
<i>Respecting Delivery Acceptance Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{i \in R} X_{i,j,k} m_{i,j} \le a_j & \forall j \in S, k \in W
\end{array}
\end{equation}

\
<i>Respecting Truck Constraints</i>

\begin{equation}
\begin{array}{11}
\sum_{j \in S} X_{i,j,k}m_{i,j} \le C_iY_{i,k} & \forall i \in R, k \in W
\end{array}
\end{equation}



\
<b>General Representations</b>

\
<i>Sets</i>
* <i>R</i>: Set of routes
* <i>S</i>: Set of stores
* <i>W</i>: Set of days in a week

\
<i>Parameters</i>
\
$
\begin{equation} 
\begin{array}{11}
d_j & \text{: Weekly demand of store j,} & j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
a_j & \text{: Delivery acceptance limits for store j,} & j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
m_{i,j} & \text{: Dummy variable to indicate 1 if store j is along route i and 0 otherwise,} & i \in R, j \in S  
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
c_{i,k} & \text{: Cost of  truck on each route i on day k } 
\end{array}
\end{equation}
$

$
\begin{equation} 
\begin{array}{11}
C_{i} & \text{: Full capacity of truck on route i on each day  } 
\end{array}
\end{equation}
$

\
<i>Variables</i>
$
\begin{equation} 
\begin{array}{11}
X_{i,j,k} & \text{: Units on each route i to be delivered to a store j on day k} \\
Y_{i,k} & \text{: If route i is used on day k}
\end{array}
\end{equation}
$

\
<i>Entities</i>
* <i>i</i>: Route in R
* <i>j</i>: Store in S
* <i>k</i>: Day in W


In [14]:
#Importing Packages

from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

In [15]:
#Importing Data

routes = pd.read_excel("routes.xlsx").to_numpy()
print(routes)

demand = pd.read_excel("data-demands-deliveyconstraint.xlsx").to_numpy()
print(demand)


[[1 'S1' nan]
 [2 'S2' nan]
 [3 'S3' nan]
 [4 'S4' nan]
 [5 'S5' nan]
 [6 'S6' nan]
 [7 'S7' nan]
 [8 'S8' nan]
 [9 'S9' nan]
 [10 'S10' nan]
 [11 'S11' nan]
 [12 'S12' nan]
 [13 'S13' nan]
 [14 'S14' nan]
 [15 'S15' nan]
 [16 'S16' nan]
 [17 'S17' nan]
 [18 'S18' nan]
 [19 'S1' 'S10']
 [20 'S3' 'S12']
 [21 'S14' 'S8']
 [22 'S7' 'S5']
 [23 'S13' 'S4']
 [24 'S15' 'S18']]
[['S1' 33.88305584537084 15]
 ['S2' 54.998014976826894 25]
 ['S3' 30.82250049022967 18]
 ['S4' 60.76642143604292 22]
 ['S5' 62.414188052054804 22]
 ['S6' 81.78132836111571 24]
 ['S7' 62.70057409550929 27]
 ['S8' 55.68341806345825 30]
 ['S9' 83.6263353931504 30]
 ['S10' 40.871401787809354 30]
 ['S11' 63.15363053970106 36]
 ['S12' 51.77423910708216 40]
 ['S13' 80.38611856493507 50]
 ['S14' 55.131182993389196 50]
 ['S15' 93.78548110609705 65]
 ['S16' 138.9411964720462 99]
 ['S17' 150.0 108]
 ['S18' 21.56053569869876 65]]


In [16]:
#Demand
d = demand[:,1]

#Delivery constraints
dc = demand[:,2]

#Route matrix
#Store names 
snames = demand[:,0]

count = 1
mat = np.zeros(24)
for i in snames:
    routestore1 = (routes[:,1] == str(i)) * 1
    routestore2 = (routes[:,2] == str(i)) * 1
    mainroutestore = routestore1 + routestore2
    mat = np.row_stack((mat, mainroutestore))
    #mat = np.concatenate((mat, mainroutestore),axis=1)
    
mat = np.array(mat, dtype='int16')[1:,:].transpose((1,0))

print(mat)


[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]]


In [19]:
#Shipping costs
c = np.full((24, 18), 100)
c

#Supply constraints (truck capacity)
t = np.full((24,7), 24)
t

array([[24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24],
       [24, 24, 24, 24, 24, 24, 24]])

In [24]:
#Transportation Problem
m4 = Model('transportation')

#Variables
#Variables are in proportion
var = m4.addVars(24,18,7)
yvar = m4.addVars(24,7,vtype = GRB.BINARY)

#Objective
m4.setObjective(sum(yvar[i,k] for i in range(24) for k in range(7)), GRB.MINIMIZE)

#Weekly Demand
for j in range(18):
    m4.addConstr(sum(var[i,j,k]*mat[i,j] for i in range(24) for k in range(7)) == d[j])
    
#Delivery Constraints
for j in range(18):
    for k in range(7):
        m4.addConstr(sum(var[i,j,k]*mat[i,j] for i in range(24)) <= 0.6*dc[j])

#Supply constraint
for i in range(24):
    for k in range(7):
        m4.addConstr(sum(var[i,j,k]*mat[i,j] for j in range(18)) <= t[i,k]*yvar[i,k])
        
#Solving the optimization problem
m4.optimize()

#Printing the optimal solutions obtained
print("Optimal Solutions:")
for i, val in var.items():
    if val.getAttr("x") != 0:
        print("Number of units from route %g to store %g on day %g:\t %g " %(i[0]+1, i[1]+1, i[2]+1, val.getAttr("x")))
        
        
#Printing y
for i, val in yvar.items():
    print("Run route %g on day %g:\t %g " %(i[0]+1, i[1]+1, val.getAttr("x")))
    
print(yvar)
 


Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 312 rows, 3192 columns and 798 nonzeros
Model fingerprint: 0x633d34aa
Variable types: 3024 continuous, 168 integer (168 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 2e+02]
Presolve removed 275 rows and 3143 columns
Presolve time: 0.01s
Presolved: 37 rows, 49 columns, 112 nonzeros
Variable types: 28 continuous, 21 integer (21 binary)

Root relaxation: objective 6.088136e+01, 38 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   60.88136    0    6          -   60.88136      -     -    0s
H    0     0                      63.0000000   60.88136  3.36%     -    0s
H    0     0          

In [26]:
for i in range(24):
    for k in range(7):
        print(yvar[i,k].getAttr("x"))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
1.0
-0.0
1.0
1.0
-0.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
1.0
1.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
-0.0
1.0
-0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
1.0
1.0
1.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
-0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
-0.0
1.0
-0.0
-0.0
-0.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
-0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
